In [1]:
import pandas as pd
print('haha')
df1 = pd.read_excel('messagesample1.xlsx') ##29767 label 1
df2 = pd.read_excel('messagesample0.xlsx') ###select the first 30000
df2 =df2.iloc[:30000]
df = pd.concat([df1,df2])
print(len(df))

haha
59767


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import sklearn.feature_extraction
import sklearn.naive_bayes as nb
import sklearn.externals.joblib as jl
import jieba
import os
from elasticsearch import Elasticsearch
import jieba.analyse
import jieba.posseg as pseg
from sklearn.svm import SVC 
from sklearn.feature_extraction.text import TfidfVectorizer
import csv
import sklearn.externals.joblib as jl
import pickle

In [3]:
a = 0
b  = 0

for index, row in df.iterrows():
    if row['code_str'] == 'OK-denied':
        a = a+1
    else:
        b =b+1
print(a)        
print(b) 

29898
29869


In [4]:
def fenci(summary):
    nlp_features_list = ""
    nlp_features = pseg.cut(summary)
    for w in nlp_features:
        nlp_features_list = nlp_features_list + " " + w.word 
    return nlp_features_list 
df['label'] = 0
df.loc[df['code_str']=='OK-denied','label'] =1
print(df.groupby(['label']).count() )


       server_id  user_id  date_time  content  def_type  def_lan  def_code  \
label                                                                        
0          29869    29869      29869    29868     29869    29869     29869   
1          29898    29898      29898    29897     29898    29898     29898   

       code_id  code_str  category  score  
label                                      
0        29869     29869     29869  29869  
1        29898     29898     29898  29898  


In [5]:
raw_data = []
category_list = []
##loop data frame
for index, row in df.iterrows():
    try:
        raw_data.append(fenci(row['content']))
        category_list.append(row['label'])
    except Exception:
        print("%s %s" %(index,row['content'] ))
print(len(raw_data))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/5p/p0cynt1n6kg2v_06_3svvm0m0000gn/T/jieba.cache
Loading model cost 0.881 seconds.
Prefix dict has been built succesfully.


27203 nan
218 10
395 9
638 1
647 1
759 2.2
1249 11.57
1269 914
1390 3.42
1391 1
1424 1
1425 1
1427 1
1445 129
1456 342
1480 342
1568 7500
1790 3.47
1796 3.26
2114 3.11
2119 6.08
2131 15736537223
2204 43135
2241 510390
2322 0
2349 0
3123 1.13
3133 1.55
3134 1.34
3403 201
3419 04:09:00
3425 519
3451 281
3604 9
3619 1
3628 34
3644 26
3668 1
3825 29
3838 19
3932 03:41:00
3939 5.45
3952 555
4087 435.532
4187 40
4294 5
4370 666
4510 19
4665 1
4669 1
4670 1
4723 1
4739 1
4742 11
4745 1
4747 11
4787 1
5052 1
5077 804.339
5272 13736858257
5302 11.31
5398 6.53
5503 4.56
5742 6
6043 1
6145 9
6156 384.765
6268 6.16
6291 7.16
6422 4.31
6471 13
6738 5.15
6759 5.5
6780 1
6787 1
6792 3
6829 10
7243 13149792679
7303 1
7322 2
7345 3
7571 2.59
7584 323
7763 17640451567
7840 7.56
7926 7.56
8054 313.289
8270 494.73
8358 10
8630 668
8745 720387
8760 713384
8892 2596
8900 8
8947 15
9055 13365305255
9180 13365305255
9278 8
9577 486651
9582 1
9689 68
9734 0.37
9757 0.24
9830 0.45
9885 13721626372
9957 1
9966 2

In [7]:
label_count = [[x,category_list.count(x)] for x in set(category_list)]
print(label_count)

[[0, 29525], [1, 29897]]


In [8]:
print("start training")
svclf = SVC(kernel = 'linear')#default with 'rbf' linear
fh =  TfidfVectorizer(sublinear_tf = True,  max_df = 0.5); 
gnb = nb.MultinomialNB(alpha = 0.01)
X = fh.fit_transform(raw_data)
pickle.dump(fh.vocabulary_, open('vocabulary', 'w'))
print("dump vocabulary")
svclf.fit(X,category_list)  
jl.dump(svclf,'final.pkl')
print("end training")

start training
dump vocabulary
end training
